In [1]:
import pandas as pd
import numpy as np
import numpy.random as nr

from collections import Counter

# 

In [2]:
raw = pd.read_csv("example.csv", index_col=0)
raw["ENTITY"] = raw.index
raw.index = raw.index.map(str) +"_"+ raw["TIME"].map(str)
raw.rename(columns={"TIME.1":"TIME"})
del raw.index.name
n = Counter(raw.DATASET)

In [3]:
raw1 = raw[raw.DATASET == 1][::]
raw2 = raw[raw.DATASET == 2][::]
raw3 = raw[raw.DATASET == 3][::]

datasets = [raw1, raw2, raw3]
means1 = [10, 12, 16]
stds1 = [3, 3, 2]

means2 = [10, 10, 10]
stds2 = [1, 1, 1]

means3 = [10, 20, 30]
stds3 = [3, 3, 2]

means4 = [20, 21, 20]
stds4 = [4, 5, 4]

degs = [2,3,4]

vals1 = ["a", "b", "c"]
probs1 = [[0.2, 0.7, 0.1], [0.3, 0.3, 0.4], [0.8, 0.1, 0.1]]

vals2 = [1, 2, 3]


batches1 = [0, 0, 5]
b_means1 = [20, 20, 20]
b_stds1 = [2, 1, 1]

batches2 = [0, 0, 2]
b_means2 = [20, 20, 20]
b_stds2 = [1, 1, 1]

#longitudinal
long_m = [[10, 12, 15, 17, 19, 22, 25, 27, 29, 33], 
         [12, 12, 16, 19, 23, 19, 28, 30, 31, 33],
         [10, 12, 14, 16, 18, 20, 22, 24, 26, 28]]
long_s = [[2.1, 1.3, 1, 1.5, 1.7, 2, 2.5, 3, 3, 3.3],
         [2.2, 1.3, 1, 1.5, 1.7, 2, 2.5, 3, 3, 3.3],
         [2.1, 1.3, 1, 1.5, 1.7, 2, 2.5, 3, 3, 3.3]]

In [4]:
def fill_long_feat(feat, datasets, time, means, stds):
    
    times = datasets[0][time].unique()
    
    for df, mean, std in zip(datasets, means, stds):

        for t, m, s in zip(times, mean, std):
            inds = df[df[time]==t].index
            n_vals = nr.normal(m, s, size=len(inds))
            df.loc[inds, feat] = n_vals
            

fill_long_feat("LONG1", datasets, "TIME", long_m, long_s)

# Generate normally distributed features

In [5]:
def fill_nd_feats(feat, datasets, means, stds):
    for df, mean, std in zip(datasets, means, stds):
        df[feat] = nr.normal(mean, std, size=len(df))
        
fill_nd_feats("NDSIG1", datasets, means1, stds1)
fill_nd_feats("NDSIG2", datasets, means3, stds3)
fill_nd_feats("NDNON1", datasets, means2, stds2)
fill_nd_feats("NDNON2", datasets, means4, stds4)

# Chi² feats

In [6]:
def fill_chi_feats(feat, datasets, degs):
    for df, deg in zip(datasets, degs):
        df[feat] = nr.chisquare(deg, size=len(df))

fill_chi_feats("CHIFEAT", datasets, degs)

# Categorical feats

In [7]:
def fill_cat_feats(feat, datasets, vals, probs=None):
    if probs:
        for df, prob in zip(datasets, probs):
            df[feat] = nr.choice(vals, p=prob, size=len(df))
    else:
        for df in datasets:
            df[feat] = nr.choice(vals, size=len(df))
        
fill_cat_feats("CATSIG1", datasets, vals1, probs1)
fill_cat_feats("CATNON2", datasets, vals2)

# Batch

In [8]:
def fill_nd_feats(feat, datasets, means, stds, batches):
    for df, mean, std, batch in zip(datasets, means, stds, batches):
        df[feat] = nr.normal(mean, std, size=len(df))
        # add batch effect
        df[feat] = df[feat] + batch

fill_nd_feats("BATCH1", datasets, b_means1, b_stds1, batches1)
fill_nd_feats("BATCH2", datasets, b_means2, b_stds2, batches2)

# Longitudinal

In [9]:
# introduce longitudinal increade or decrease


# Combine

In [10]:
pd.concat(datasets)

,TIME,DATASET,FEAT1,ENTITY,LONG1,NDSIG1,NDSIG2,NDNON1,NDNON2,CHIFEAT,CATSIG1,CATNON2,BATCH1,BATCH2
1_0,0,1,658,1,13.556015,13.285941,15.575843,10.129392,15.492019,0.742549,a,1,18.437925,17.454807
1_2,2,1,543,1,10.043433,6.771134,12.027988,10.973203,27.797558,5.647079,c,2,21.476235,20.295356
1_4,4,1,520,1,12.838797,11.416374,14.471488,10.219096,26.777205,0.118491,b,1,18.755405,20.238939
1_6,6,1,563,1,16.796939,11.868754,1.203594,9.629253,20.788811,1.292498,b,3,22.047700,22.244320
1_8,8,1,389,1,18.386617,12.560026,10.017828,10.948651,15.101437,4.904571,b,2,15.989785,20.104390
1_10,10,1,371,1,22.285776,12.120038,14.692696,8.779462,20.330745,0.668617,b,1,20.668656,21.643829
2_0,0,1,500,2,11.399102,11.172722,9.779726,10.419645,21.193427,3.503380,a,2,18.637006,18.423284
2_2,2,1,419,2,12.302206,8.787648,8.574507,8.390331,15.397635,2.620705,b,3,18.288263,21.096608
2_4,4,1,431,2,14.651231,9.892890,9.904105,11.115166,10.914600,0.577248,b,3,19.789707,20.013330
2_6,6,1,285,2,14.270999,9.905725,9.109659,11.066687,21.201133,2.510565,c,3,23.825235,20.715190


In [11]:
pd.concat(datasets).to_csv("simulated.csv")